In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
tmdb_file = "Resources/movies_metadata.csv"
tmdb_df = pd.read_csv(tmdb_file, sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
tmdb_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


### Transform tmdb DataFrame

In [16]:
# Create a filtered dataframe from specific columns for TMDB_Data table
tmdb_data_cols = ["original_title", "original_language", "runtime","release_date"]
tmdb_data_transformed= tmdb_df[tmdb_data_cols].copy()

# Rename the column headers
tmdb_data_transformed = tmdb_data_transformed.rename(columns={"original_title": "Movie_title",
                                                          "original_language": "Original_Language",
                                                          "runtime": "Runtime",
                                                          "release_date": "Release_date"})

# Clean the data by dropping duplicates, null and setting the index
tmdb_data_transformed.drop_duplicates("Movie_title", inplace=True)
tmdb_data_transformed.set_index("Movie_title", inplace=True)
tmdb_data_transformed['Release_date'] = pd.to_datetime(tmdb_data_transformed['Release_date'], errors='coerce')
tmdb_data_transformed.dropna(how='any')
#tmdb_data_transformed=tmdb_data_transformed[tmdb_data_transformed.Runtime != 'null']

tmdb_data_transformed.head()

,Original_Language,Runtime,Release_date
Movie_title,,,
Toy Story,en,81.0,1995-10-30
Jumanji,en,104.0,1995-12-15
Grumpier Old Men,en,101.0,1995-12-22
Waiting to Exhale,en,127.0,1995-12-22
Father of the Bride Part II,en,106.0,1995-02-10


In [5]:
# Create a filtered dataframe from specific columns for TMDB_Score table
tmdb_score_cols = ["original_title", "vote_average"]
tmdb_score_transformed= tmdb_df[tmdb_score_cols].copy()

# Rename the column headers
tmdb_score_transformed = tmdb_score_transformed.rename(columns={"original_title": "Movie_title",
                                                          "vote_average": "TMDB_Score"})

# Clean the data by dropping duplicates, null and setting the index
tmdb_score_transformed.drop_duplicates("Movie_title", inplace=True)
tmdb_score_transformed.set_index("Movie_title", inplace=True)
tmdb_score_transformed.dropna(how='any')
tmdb_score_transformed.head()

,TMDB_Score
Movie_title,
Toy Story,7.7
Jumanji,6.9
Grumpier Old Men,6.5
Waiting to Exhale,6.1
Father of the Bride Part II,5.7


In [6]:
# Create a filtered dataframe from specific columns for TMDB_Vote_Count table
tmdb_vote_cols = ["original_title", "vote_count"]
tmdb_vote_transformed= tmdb_df[tmdb_vote_cols].copy()

# Rename the column headers
tmdb_vote_transformed = tmdb_vote_transformed.rename(columns={"original_title": "Movie_title",
                                                          "vote_count": "Vote_Count"})

# Clean the data by dropping duplicates, null and setting the index
tmdb_vote_transformed.drop_duplicates("Movie_title", inplace=True)
tmdb_vote_transformed.set_index("Movie_title", inplace=True)
tmdb_vote_transformed.dropna(how='any')
tmdb_vote_transformed.head()

,Vote_Count
Movie_title,
Toy Story,5415
Jumanji,2413
Grumpier Old Men,92
Waiting to Exhale,34
Father of the Bride Part II,173


### Create database connection

In [22]:
connection_string = "postgres:Krishna39@localhost:5432/Movies"
engine = create_engine(f'postgresql://{connection_string}')

In [23]:
# Confirm tables
engine.table_names()

['Netflix',
 'Wiki_Plots',
 'IMDB_Data',
 'IMDB_Score',
 'IMDB_Vote_COunt',
 'TMDB_Data',
 'TMDB_Score',
 'TMDB_Vote_Count']

### Load DataFrames into database

In [19]:
tmdb_data_transformed.to_sql(name='TMDB_Data', con=engine, if_exists='append', index=True)

In [24]:
tmdb_score_transformed.to_sql(name='TMDB_Score', con=engine, if_exists='append', index=True)

In [25]:
tmdb_vote_transformed.to_sql(name='TMDB_Vote_Count', con=engine, if_exists='append', index=True)